In [1]:
#All tools and imports
import io
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import load_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input

import anvil.media
import anvil.server

In [2]:
#Load chosen model (MobilenetV3)
model = tf.keras.models.load_model('./models/mn_model.h5')

In [3]:
#Function to predict user image
def image_prediction(file):
   
    #Image path
    image_path = file

    #Processing image
    image = keras.preprocessing.image.load_img(image_path, target_size= (224,224))
    image_arr = img_to_array(image)
    image_arr = np.expand_dims(image, axis=0)

    #Predicting User images pic
    result = model.predict(image_arr)

    prediction = "Most Likely Species is "
    
    if result[0][0] > 0.5:
        prediction = prediction + "Bald Eagle"
        result_score = ((result[0][0]) * 100)
    if result[0][1] > 0.5:
        prediction = prediction + "Golden Eagle"
        result_score = ((result[0][1]) * 100)
    if result[0][2] > 0.5:
        prediction = prediction + "Red Tailed Hawk" 
        result_score = ((result[0][2]) * 100)
    
    return prediction, result_score

In [4]:
#Connecting to Anvil Server
anvil.server.connect("server_LEBLS4DZDUAL75CBE7G7RZPN-CUYK6V63QEF3H45P")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [5]:
#Funtion to connect to Anvil web app
@anvil.server.callable
def predict_image(file):
    with anvil.media.TempFile(file) as filename:
        
        img = io.BytesIO(file.get_bytes()) 
        prediction, result_score = image_prediction(img)
        
    return(prediction, result_score)        

In [6]:
#Web app address
#https://jennifercapstone2023.anvil.app/